In [1]:
from collections import defaultdict
import itertools
from matplotlib import pyplot as plt
import math
import numpy as np
import operator
import pandas as pd
import pickle
import random
import re
import scipy
import string
from tqdm import tqdm_notebook


import src.knntools as knntools
import src.postprocess as postprocess
import src.preprocess as preprocess
import src.tfidftools as tfidftools
import src.tools as tools
import src.recencytools as recency
import src.scoring as scoring
import src.textembeddingtools as texttools
import src.graphwordstools as graphtools

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
path_to_data = 'data/'

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(
    path_to_data + 'training_info.csv', sep=',', parse_dates=True, header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

test_info = pd.read_csv(path_to_data + 'test_info.csv',
                        sep=',', parse_dates=True, header=0)
path_to_results = 'results/'

In [3]:
training_body_dict = preprocess.body_dict_from_panda(training_info)
test_body_dict = preprocess.body_dict_from_panda(test_info)

In [5]:
training_email_ids_per_sender = preprocess.get_email_ids_per_sender(training)
test_email_ids_per_sender = preprocess.get_email_ids_per_sender(test)

In [6]:
training_token_dict = texttools.get_token_dict(training_body_dict)
test_token_dict = texttools.get_token_dict(test_body_dict)

## Compute average length of doc in tokens

In [7]:
training_doc_lengths_dic, training_average_doc_len = texttools.get_doc_length_info(training_token_dict)

In [8]:
training_idf_dic, training_idf_words = tfidftools.get_idf_dic(training_token_dict)

In [9]:
print(len(training_idf_dic))

7702


In [13]:
# %%time
twidf_matrix, twidf_mids = graphtools.get_twidf_matrix(training_token_dict, training_doc_lengths_dic,
                                                        training_average_doc_len, training_idf_dic,
                                                        training_idf_words, 0.003)


# with open('variables/twidf_matrix', 'wb') as outfile:
#     pickle.dump(scipy.sparse.csr_matrix(twidf_matrix), outfile)
    
# with open('variables/twidf_mids', 'wb') as outfile:
#     pickle.dump(twidf_mids, outfile)

In [14]:
test_vectors = graphtools.get_twidf_vectors_from_tokens(training_idf_dic, training_idf_words,
                                                       test_token_dict, training_average_doc_len)

In [15]:
test_recipient_scores = knntools.compute_twidf_similarity_scores(twidf_matrix, twidf_mids,
                                                                 test_vectors, training_info,
                                                                 nb_similars=50)

2306/|/ 98%|| 2306/2362 [02:44<00:03, 14.01it/s]


In [16]:
twidf_predictions = knntools.similar_dic_to_standard(test_recipient_scores, nb_recipients=100)

In [17]:
path_to_ranks = 'ranks_test/twidf-knn-k-{nb_neighbors}-rec'.format(nb_neighbors=50)
with open(path_to_ranks, 'wb') as infile:
              pickle.dump(twidf_predictions, infile)

In [21]:
postprocess.write_results_ranked(twidf_predictions, path_to_results, 'twidf-knn-50.txt')